### Data Prep

In [2]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('student_data.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.iloc[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

In [5]:
features.shape

(360, 6)

### Train Neural Network
#### Define activation function

In [6]:
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

#### Set parameters

In [20]:
# Use same seed to make debugging easier
np.random.seed(42)

# Initialize weights
n_records, n_features = features.shape
weights = np.random.normal(scale = 1/n_features**.5, size = n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

#### Training
Train Neural Network and print the result every 1000/10 iteration

In [21]:
last_loss = None
for e in range(epochs):
    # For each iteration:
    # Step 1 - Calculate total weight change
    del_w = np.zeros(weights.shape) # initialize total weight change
    for x, y in zip(features.values, targets): # Loop through all records, x is the input, y is the target
        h = np.dot(weights, x) # the weighted sum
        output = sigmoid(h) # the prediction y_hat
        error = y - output # the error
        error_term = error*output*(1-output) # the error term
        # Calculate the change in weights for this sample and add it to the total weight change
        del_w += learnrate * error_term * x
    # Step 2 - Update weights
    weights += del_w / n_records
    
    # Print out the mean square error on the traning set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets)**2)
        if last_loss and last_loss < loss:
            print("Train loss: ",loss, " WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

Train loss:  0.26276093849966364
Train loss:  0.20928619409324895
Train loss:  0.20084292908073417
Train loss:  0.1986215647552789
Train loss:  0.19779851396686018
Train loss:  0.19742577912189863
Train loss:  0.19723507746241065
Train loss:  0.19712945625092465
Train loss:  0.19706766341315077
Train loss:  0.19703005801777368


#### Calculate accuracy on test data

In [22]:
test_out = sigmoid(np.dot(features_test, weights))
predictions = test_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Prediction accuracy: 0.725
